In [2]:
import numpy as np

In [84]:
def is_prime(n):
    s = np.sqrt(n+1.0)
    for i in xrange(2, int(s+1)):
        if n % i == 0:
            return False
    return True

In [102]:
def _fed_tau_internal(num_time_steps, scale, stability, reorder_flag):
    tau = np.zeros(num_time_steps)
    
    # saving future computations
    denom = 1./(4.*num_time_steps + 2.)
    num = scale * stability / 2.
    
    # setting up tau vector
    for k in xrange(num_time_steps):
        h = np.cos(np.pi * ( (2.*k + 1.) * denom ) )
        tau[k] = num / (h*h)
    
    if reorder_flag == True:
        permuted_indices = get_permutation(num_time_steps)
        tau = tau[permuted_indices]
            
    return num_time_steps, tau

In [103]:
def get_permutation(num_time_steps):
    if num_time_steps < len(kappa_lookup): # FED_MAXKAPPA in the C code
        kappa = kappa_lookup[num_time_steps]
    else: 
        kappa = int(num_time_steps/4)

    # find the prime on which to permute
    prime = num_time_steps + 1
    while not is_prime(prime):
        prime += 1

    # perform the permutation!
    indices = []
    k = 0
    for l in xrange(num_time_steps):
        index = (k+1)*kappa % prime - 1.
        while index >= num_time_steps:
            k += 1
            index = (k+1)*kappa % prime - 1.
        indices.append(int(index))
        k+=1
        
    return indices

In [104]:
def fed_tau_by_process_time(stop_time, num_cycles, stability, reorder_flag):
    cycle_time = stop_time/num_cycles
    N, tau = fed_tau_by_cycle_time(cycle_time, stability, reorder_flag)
    return N, tau

In [105]:
def fed_tau_by_cycle_time(stop_time, stability, reorder_flag):
    num_time_steps = int(np.ceil(np.sqrt(3.*stop_time / stability + 0.25)
                                 -0.5 - 1.e-8) + 0.5)
    scale = 3.*stop_time / (stability*float(num_time_steps*(num_time_steps
                                                            +1)))
    N, tau = _fed_tau_internal(num_time_steps, scale, stability, 
                               reorder_flag)
    return N, tau

In [106]:
import unittest

class TestFED(unittest.TestCase):

    def test_fed_tau_by_process_time(self):
        N, tau = fed_tau_by_process_time(500.0, 5, 0.5, True)
        self.assertEquals(N, np.size(tau))
        expected = np.array([0.283858,0.456195,1.327985,60.901379,0.250257,
                             0.298989,0.516557,1.775248,0.252327,0.318043,
                             0.594955,2.517796,0.256534,0.341871,0.698839,
                             3.885568,0.263024,0.371646,0.840030,6.841508,
                             0.272022,0.408991,1.038190,15.288122,])
        #blah = [     0.2838577032089233,     0.4561947584152222,     1.3279851675033569,    60.9013786315917969,     0.2502571046352386,     0.2989892661571503,     0.5165565013885498,     1.7752484083175659,     0.2523265779018402,     0.3180425763130188,     0.5949546694755554,     2.5177960395812988,     0.2565344870090485,     0.3418713808059692,     0.6988388895988464,     3.8855679035186768,     0.2630237936973572,     0.3716463446617126,     0.8400301933288574,     6.8415083885192871,     0.2720217704772949,     0.4089909493923187,     1.0381901264190674,    15.2881221771240234,]
        #expected = np.array(blah)
        #for x, y in zip(tau, expected):
            #print 'tau', x, 'ex', y, 'diff', x-y
        # Lists differ at index 3: 60.9014423697, 60.901379
            
        for i, (actual_x, expected_x) in enumerate(zip(tau, expected)):
            msg = "Lists differ at index %d: %s, %s" % (i, actual_x, expected_x)
            self.assertAlmostEqual(actual_x, expected_x, places=4, msg=msg)
        
        
unittest.main(argv=[''], exit=False)

F
FAIL: test_fed_tau_by_process_time (__main__.TestFED)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-106-36df332e16a4>", line 21, in test_fed_tau_by_process_time
    self.assertAlmostEqual(actual_x, expected_x, places=4, msg=msg)
AssertionError: Lists differ at index 3: 60.9014423697, 60.901379

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (failures=1)


In [64]:
print type(np.pi)

<type 'float'>


In [30]:
kappa_lookup = [
     1,     1,     1,     1,     2,     2,     5,     4,     4,     3,
     2,     5,     4,     9,     3,    14,    13,     7,     4,    12,
    18,    13,     6,    17,     6,     5,     4,    20,    22,     5,
    11,    16,    27,    10,    10,    15,    15,    28,    11,    18,
    28,    26,     8,    12,    19,    13,    13,    14,    32,    36,
    29,     8,    23,    40,    12,    10,    26,    34,    40,    37,
     8,    51,    23,    27,    29,    15,    28,    12,    61,     6,
    17,    30,    28,    47,    67,    47,    67,    14,    57,    66,
    54,    25,    63,    15,    36,    53,    61,    16,    61,    22,
    20,    66,    20,    86,    42,    23,    89,    74,    13,    12,
    18,    19,    47,    86,    19,    23,    24,    14,    26,    20,
    71,    87,    40,    27,    26,    53,    70,    89,    49,    87,
    75,    34,    68,    17,   108,    72,    93,    34,    59,    20,
    30,    66,    99,    97,    19,    51,   104,    22,    36,    52,
    41,    29,    90,    39,    45,    70,    94,    84,    40,    99,
    31,    17,    27,    64,    64,    65,    88,    29,    91,    87,
    30,    45,    63,    17,    97,    65,    20,    50,    76,    70,
    66,    24,    92,    78,    22,    26,   132,    31,    99,    48,
   154,    90,    71,   135,    35,   108,    50,   120,    69,    61,
    30,    26,    26,    37,    57,   108,    80,    17,   121,    95,
    43,   149,    95,   144,    27,    48,    49,    24,    95,    18,
    61,    90,    48,    61,   171,    97,    69,   100,   154,    19,
    98,    50,   132,   167,   122,    80,   178,   166,   134,    71,
   143,   109,    95,   175,    54,    29,   100,    49,   173,    29,
    99,    48,    70,    75,    30,   206,   191,    37,   116,    19,
   112,    74,   221,   104,   164,    97,    30,    72,   110,   205,
    60,   161,   152,    21,   245,    75,   113,    40,   194,    41,
    75,    32,   202,   101,    54,    50,   168,    67,   119,    86,
    27,    43,   111,   255,   240,   226,    52,   199,    66,   102,
   103,    25,    66,    45,    66,    42,    83,    67,    66,    21,
   169,   264,   193,    65,   172,    86,   233,   120,    40,   238,
   122,    49,    83,    54,   220,   202,   129,    42,   179,    26,
   195,    50,   242,   211,   210,    85,   118,   196,    52,   183,
    58,    60,   284,   216,   120,   229,    60,    83,    64,    74,
    55,    48,    55,    30,    47,   191,    93,    68,   211,    34,
   152,    24,   104,   225,   277,    61,    82,   275,   194,    68,
   216,    69,   111,   164,    78,   205,   327,    88,   256,   283,
   110,   272,   242,    39,   161,   146,   343,   292,   137,   158,
   168,   181,   295,   120,   298,   181,   211,    91,   218,   148,
   232,   164,   143,    52,   222,    71,   174,    38,   288,    55,
   359,   252,   113,    78,   107,    75,   235,   147,    44,   226,
   117,   112,    73,   170,   160,    90,   151,   194,   124,    31,
    95,    59,   351,    63,   138,   273,   135,   193,   303,    58,
   336,    92,   154,   179,    58,   180,   123,    90,   336,    80,
   359,   241,   364,   176,   252,   356,   277,    66,   197,   214,
   243,   106,   178,    71,   205,   346,   333,   100,   134,   111,
   139,   284,   281,   195,   164,   274,   346,   262,   293,   101,
   271,    62,   347,   363,   105,   312,   421,    76,   176,   144,
   338,   104,    74,    47,   380,   312,    66,   347,   101,    48,
    87,    74,   308,    61,   461,    93,    90,    66,   418,    43,
   317,    60,   113,   142,   151,   223,   325,   149,   427,   122,
   154,   117,   118,    55,   108,   117,   198,   218,   378,   420,
   381,    79,   287,   456,   202,   156,   167,   145,   229,   126,
   167,    84,   381,   115,   411,    44,   164,   228,    87,   220,
   145,   389,   192,   148,   302,   203,   306,    85,   394,    45,
    82,   240,   459,   164,   466,   115,    67,   121,   473,   317,
   436,   395,   412,   238,   461,   137,   461,   399,   322,   337,
   118,    56,    92,    76,   498,   104,   214,   175,   133,    72,
   186,    75,   324,    71,   239,   113,   155,    88,   256,    95,
   181,   469,   251,   213,   103,   275,   277,   474,   489,    49,
   538,   215,   169,   175,   105,   460,   218,   172,   344,   158,
   171,   177,   346,    98,   503,   114,   217,   146,   252,   397,
   483,    77,   282,    76,   535,   466,   274,   518,   392,   100,
   341,   277,   545,   283,   462,    63,   494,    85,   231,   232,
   494,   120,   466,   153,    87,   300,   449,   480,   509,    75,
   170,   396,   292,   144,   286,   417,   536,    78,   295,   116,
   408,    88,   379,    54,   188,   151,   278,   375,   545,   180,
   120,    55,    60,   212,   264,   487,   569,   624,   282,   397,
   243,   126,   524,   134,   557,   245,   245,   265,   496,    52,
   428,    83,    91,   517,   130,   243,   130,   104,   376,    91,
   307,   127,   268,   186,   639,   104,   126,   437,   122,   420,
   459,   618,   155,    94,   198,    85,   557,   495,   517,   456,
   197,   456,   560,    86,   201,   405,   127,   198,   119,   269,
   160,   432,   221,   433,   161,   509,   339,   214,   136,   576,
   201,    98,   580,   208,   539,   120,   479,   175,   330,   285,
   405,   479,   345,   654,   204,   317,   637,   140,   660,   448,
   449,   277,   292,   131,    92,   343,   107,   142,   284,    93,
   176,   134,   167,   441,   188,   103,   212,   671,   435,   582,
   229,   441,   149,   208,   559,   566,   548,   232,   182,   146,
   141,   728,   179,   302,   561,   352,   302,   563,   583,   126,
   175,   332,   433,   501,   585,   496,   516,   467,   509,   248,
   656,    87,   155,   668,   334,   472,   187,   225,   558,   220,
   187,   155,   244,   259,   655,   673,   115,   156,   467,   192,
   298,   462,   354,   124,   678,   342,   319,   183,   593,   400,
   372,   354,   239,   703,   549,   354,   549,   228,   361,   236,
   156,   109,   742,   178,   602,   222,   255,   236,   241,   178,
   320,    98,   499,   620,   761,   136,   256,   156,   568,   183,
   618,   139,   707,   785,   383,   199,   143,    79,   521,    71,
   774,   487,   636,   415,   184,   191,   374,   251,   343,    78,
   166,   657,   579,   766,   173,   532,   398,   196,   171,   840,
   507,   243,   343,   104,   406,   312,   398,   479,   710,   172,
   378,   239,   246,   339,   177,   433,   563,   347,   174,   166,
   434,    79,   633,   486,   222,   296,   540,   519,   537,   529,
   742,    83,   141,   408,   291,   292,   274,   555,   603,   100,
   222,   281,   347,   846,   166,    97,   391,   419,   800,   700,
   341,   798,   288,   255,   533,   417,   541,   375,   355,   182,
   661,   155,   617,   127,   533,   253,   797,   411,   371,   612,
   342,   113,   763,    92,   548,   205,   399,   103,   291,   150,
   221,   452,   150,   577,   684,   645,   443,   643,   293,   577,
   294,   137,   280,   652,   118,   440,   224,   312,   324,   137,
   186,   228,   272,   148,   130,   563,   472,   456,   547,   587,
   180,   425,   159,   149,   311,   366,   193,   309,   720,   754,
   224,   402,   213,   301,   850,   629,   549,   373,   834,   134,
   639,   250,   583,   238,   363,   821,   814,   229,   143,   181,
   226,   376,   113,   481,   606,   139,   587,   378,   219,   164,
   861,   195,   402,   307,   855,   491,   240,   388,   656,   687,
   466,   401,   394,   471,   441,   369,   881,   161,   621,   139,
   601,   164,   662,   670,   127,   126,   191,   127,   504,   255,
   188,   133,   499,   504,   387,   848,   305,   586,   143,   337,
   450,   614,   392,   178,   492,   145,   492,   641,   164,   898,
   476,   196,   307,   638,   841,   287,   259,   206,   204,   421,
   685,   325,   414,   205,   255,   403,   939,   619,   294,   340,
   316,   794,   533,   473,   883,   402,   107,   606,   691,   186,
   799,   263,   649,   169,   361,   155,   789,   155,   269,   888,
   928,   245,   898,   133,   734,   141,   890,   468,   697,   645,
   675,  1018,   272,   538,   531,   174,   954,   442,   207,   187,
   365,   307,   755,   408,   528,   727,   433,   480,   847,   327,
   485,   276,   276,   431,   721,   366,   850,   258,   694,   694,
   742,   145,   362,   266,   868,   186,   760,   505,   846,   761,
   978,   812,   365,   642,   162,   454,   503,  1073,   441,   530,
   354,   179,   426,   658,   979,   670,   436,   353,   463,   184,
   451,   375,   223,   262,   227,   421,   189,   966,  1077,   654,
   374,   704,   568,   333,  1082,   501,   444,   549,   457,   116,
   994,   230,   954,   448,   715,   151,   519,   547,   516,   118,
   696,   462,   739,   172,   345,   160,   345,   174,   474,   562,
   512,   771,   683,   703,   777,   495,   912,   337,   924,   457,
   880,  1035,   686,   969,   903,   385,   374,   916,   816,   714,
   882,   747,   891,   335,   396,   148,   173,  1060,   528,   136,
   347,   443,   677,   598,   275,   357,   699,  1058,   466,   801,
   271,   158,   376,   159,   233,   194,   166,  1098,   986,   140,
   175,   298,   721,   573,   494,   203,   929,   154,   808,   254,
  1087,   354,  1075,   962,   489,   235,  1082,   245,   210,  1149,
   171,   244,   540,   355,   518,   515,   805,   237,   734,   844,
   491,   380,   991,   797,  1056,   257,   869,   289,   798,  1125,
   202,   246,   186,  1125,   958,   406,   926,   398,   749,   556,
   256,   497,   376,   293,   148,   234,   949,   306,   112,   992,
   421,   251,   252,   403,  1054,   799,  1006,   203,   475,   331,
   182,   222,   241,  1006,  1115,   427,   817,   168,   808,   165,
   298,   227,   296,   170,   377,   492,   591,   331,   631,   224,
  1028,   498,   391,   250,   660,   394,   385,   183,   249,   327,
   150,   521,   410,   162,   544,   581,   886,  1103,   992,  1050,
   307,  1148,   798,   210,   540,   381,   397,   411,   560,   148,
   659,   399,   257,   784,   561,   524,   661,   374,   508,   212,
   874,   446,   276,   268,   425,   564,   666,   341,   425,   377,
   421,  1135,   887,   141,   257,   214,  1216,   141,   401,   230,
  1263,   196,   926,   380,  1255,   309,  1113,   191,   349,  1213,
   273,   449,   407,   253,   561,   428,   531,   896,   646,  1013,
   867,   231,  1257,   256,   232,  1088,   548,   398,   409,   879,
  1302,   201,  1053,   436,   522,   954,   172,   258,   439,   538,
   834,   173,   401,   402,  1174,   528,   289,   697,  1018,   146,
   662,   612,   260,   230,   964,   175,   357,   279,   396,   808,
   443,   849,  1118,   176,   473,   271,   878,   181,   658,   595,
   488,   532,   185,   646,   857,   449,  1246,   455,   402,   897,
  1076,   296,  1215,   333,   697,   898,   638,  1375,   652,   427,
   320,   848,   186,   561,   478,   588,   843,   939,   235,   294,
   718,   634,   570,   946,   560,   878,   595,   912,  1217,   858,
  1080,   233,   638,   591,   719,  1124,   579,   116,  1316,   424,
  1137,  1141,  1238,   407,   598,   869,   241,  1090,   294,   691,
  1225,   709,  1225,   418,   431,   433,  1406,   303,   345,  1341,
  1012,   275,   974,   619,  1112,   219,  1149,   988,   443,  1022,
   372,   587,  1317,   279,   656,   208,   911,   661,  1322,   983,
   343,   250,   616,   189,  1374,   151,   635,   447,   881,   687,
   475,   255,  1112,   475,   745,   573,   459,   123,   635,  1048,
   478,   371,   438,   620,   158,   212,   590,   615,  1429,   760,
   312,  1028,   429,  1129,   696,  1144,   935,   739,   461,   587,
   443,  1198,   642,   319,   392,   685,   879,   244,   374,   448,
   352,  1443,   972,   720,   912,   958,   447,   291,  1225,  1418,
  1248,   392,   139,  1065,   459,   392,  1372,  1045,   958,   407,
  1333,   493,   958,   326,  1390,   208,   610,   454,  1174,   663,
  1486,   218,   743,   389,  1083,   251,   710,   297,   903,   276,
   195,   927,   985,  1048,   607,   351,   659,  1091,  1448,   920,
  1258,   296,   481,   616,   668,   448,  1006,   334,   963,   363,
   227,   508,   714,   467,   381,   536,   469,   467,   523,   997,
  1340,   614,   502,  1061,   764,  1031,   758,  1531,   412,  1212,
   628,   491,   381,   938,   474,   531,   539,   632,  1486,   415,
   394,   642,   920,   680,   187,   187,   964,   237,   369,   399,
   387,   315,   542,   383,   388,   400,  1436,   214,   981,   276,
  1032,  1039,   675,   276,   384,   276,   545,   249,  1100,  1238,
  1034,   213,   986,   478,   193,   618,   740,  1446,  1444,   698,
   340,   188,   320,   534,  1269,   575,   987,   738,   408,   380,
   411,   420,   341,   432,  1649,   469,  1599,   499,  1390,   997,
   669,  1164,   484,  1311,   649,   431,  1582,   353,   490,   211,
   222,   482,  1528,   775,  1485,  1357,   825,   399,  1535,   278,
   532,   774,   491,  1183,   782,   147,  1065,   537,   537,   500,
  1442,   515,   772,   357,   714,   754,   357,   161,  1310,   727,
   726,   701,   830,   181,  1319,   200,   289,   493,   394,   238,
  1284,   692,  1085,   221,   493,   540,   820,  1364,   393,   675,
   154,   443,   409,   257,  1623,  1616,   825,   225,   737,   443,
   570,  1206,  1024,   230,  1477,   495,  1115,   221,   351,  1160,
   339,   167,   349,   688,   582,   343,   260,   232,   549,   366,
  1638,   219,   451,  1116,  1023,   739,   883,  1170,   436,   780,
   566,  1407,  1691,  1077,  1036,   232,   450,   224,   300,   697,
   369,   300,   748,  1063,   416,   621,   347,   748,  1178,   301,
  1556,   408,   606,   766,   741,  1126,  1523,   265,   427,   507,
   346,   263,  1406,   799,  1359,   188,   210,   364,  1035,   545,
   437,   554,   466,   424,   768,  1204,   316,   288,   889,   467,
  1650,  1219,   532,   859,  1716,  1264,  1355,   826,   521,   881,
   213,  1698,  1086,   207,   477,   211,   374,   231,  1374,   192,
  1268,  1552,  1152,   297,   269,  1110,   353,   322,  1115,   227,
   432,   268,  1100,   197,  1448,   238,   356,  1168,  1254,   214,
   774,   347,  1537,   434,   422,   531,   483,   531,  1541,   468,
   420,   527,   821,   419,  1471,   352,  1475,   164,  1724,   305,
  1564,   266,  1380,   909,   649,   440,   391,   306,   462,   467,
  1734,  1187,   550,   445,   570,   362,   423,   477,  1265,   169,
  1311,   605,  1151,   269,  1098,  1294,  1521,   609,  1259,  1103,
  1267,   243,  1835,  1690,  1811,  1278,   733,   537,  1486,   532,
  1220,   661,   554,   854,   903,  1672,  1161,   743,   576,   486,
   494,   486,   472,   270,   443,   549,  1596,   330,  1283,   237,
  1429,  1330,   283,   799,   365,   383,   774,   249,  1267,   319,
  1591,  1808,   546,   614,   798,   505,  1532,  1338,   664,   819,
  1217,   592,   397,   246,   943,   375,  1628,   365,  1712,  1303,
  1216,   376,   337,   767,  1661,  1346,  1142,   408,  1641,  1117,
   252,   219,  1798,   764,   628,  1635,  1869,  1641,   323,   635,
   279,   335,  1465,   254,  1309,   209,  1232,   325,  1301,  1693,
   774,   486,   512,   250,   665,  1660,   466,  1356,  1582,   220,
   455,   626,   800,   261,   459,   948,   893,  1521,   684,   758,
  1212,  1755,   806,   347,   488,  1336,  1260,   957,  1890,   288,
   810,   804,  1495,   338,  1324,  1504,   609,   403,  1779,  1411,
   404,   302,   640,   352,   903,  1630,   898,  1192,  1900,   264,
   896,   903,   680,  1396,  1584,  1584,   498,   986,   950,   652,
  1517,   495,   395,   425,  1908,   843,  1675,  1412,  1349,   515,
   787,  1283,  1253,   216,  1439,  1370,  1715,   798,  1251,   906,
  1291,  1199,  1463,   613,   630,  1834,  1380,   288,   781,   401,
   605,  1320,   856,   951,  1849,   648,   702,   520,  1299,   664,
   504,   971,   531,   476,   401,   235,  1734,  1335,   827,  1016,
   859,   513,   925,   242,  1436,   402,   792,  1033,   950,   417,
  1446,   815,  1242,  1581,   194,  1652,  1309,  1969,  1427,  1669,
  1788,  1208,   476,  1554,   366,   996,  1604,  1982,   309,   884,
  2006,  1387,   614,   399,   938,   393,  1766,   243,  1745,  1211,
   602,   427,  1853,   309,  1680,   365,   477,  1858,   423,   359,
   217,   844,  1631,   617,   946,  1400,   830,   647,  1418,   540,
   876,  2061,   854,  1984,  1822,  1886,  1286,   433,  1012,  1240,
  1416,   431,   372,   688,   506,  1807,   684,  1432,  1010,   487,
   528,   186,   557,  1384,   643,   283,   654,   343,   400,   824,
   907,  1856,  1609,   186,   633,   636,  1923,  1481,   917,  1654,
   506,   440,  1379,   862,  1933,   530,   920,   438,  1026,   929,
  1501,   352,  1706,  1450,  1716,  1059,  1950,  1850,   627,   223,
   866,   372,  1120,   752,   657,   727,   659,   283,   508,   417,
  1929,   287,  1810,  1975,   879,   912,   537,  1349,  1480,   385,
   231,  2021,  1509,   447,  1871,  1073,  1353,   318,  1544,  1729,
  1553,   755,   545,   671,  1698,   358,  1962,   500,  1545,   447,
  1364,   563,   734,   663,   213,  1738,   666,   894,   357,   378,
  1966,   366,  1372,  1379,  1009,  1476,   705,   560,   379,   713,
   472,  1581,  1379,  1343,   661,   284,   900,   320,   394,  1728,
  1539,   441,  1990,   341,   439,  1367,   877,   742,   334,   195,
  1084,   287,  1455,   900,  1501,   931,  1556,   334,  1518,   324,
  1533,  1900,  1500,  1390,   384,  1522,   917,   722,  2148,   434,
  1593,   653,  1937,  1832,   365,  1424,  1571,   389,   985,   348,
  1417,   290,  1098,   465,   529,   322,   671,   328,   970,   347,
   764,  1986,  1953,   529,   919,   765,   290,   533,  1085,   464,
   377,   887,  1443,  1394,  1091,   570,   739,   434,   874,   452,
  1536,   703,  1614,  2136,   268,   383,  1348,  1448,  1608,  1108,
   897,   668,  1596,   296,  1115,  2053,   489,   384,   398,  1832,
   204,   309,   593,   296,  1963,   275,  1764,   271,   375,   272,
   554,   904,  1790,   937,   328,   965,  2015,   412,  1051,  1629,
   715,   442,   714,  1114,   344,   336,   455,   695,  1990,  1494,
  2074,   458,   724,   916,   996,   312,   353,   943,   988,   541,
  1498,  1350,   590,  1560,  1652,   684,  1113,   729,  1519,   460,
  2004,   562,  2123,   452,  1824,  1069,  1998,  1977,   448,  1809,
  1048,   408,  1503,   226,  1620,   312,  1844,   575,   418,  1610,
   545,  1149,   572,   594,  1902,   597,   795,   492,   610,   335,
   417,   572,   590,   410,  1169,  1133,  1014,  2048,   449,   758,
  1828,  2210,   772,   402,  1942,  1154,   499,   463,  1558,   611,
  1159,   605,  1531,  1118,  1879,  1003,  2153,   942,   276,   636,
   464,   255,  2028,   462,   410,   426,   764,   489,  1097,  2183,
  1025,  1498,  1821,   740,   393,   608,  1907,   577,  1632,   309,
  1965,   712,  1258,   745,   735,   851,  1203,   619,  1237,   964,
   567,  2194,   795,  1206,   508,  1545,   407,  1496,   364,   563,
   736,  2276,  1547,  1461,  2317,  1102,  1506,   405,  1505,  2183,
   497,   565,   372,  2092,   973,   280,   834,   781,  1266,  1423,
  1121,   994,   822,   994,  1518,  1754,   716,   591,  1567,   480,
   618,   503,  1749,  1502,  2194,   612,   527,   706,  1711,   406,
   427,  1740,  2314,   750,  1127,  1218,   828,  1745,  1007,   286,
  1612,  1550,  1063,  2224,   404,  1487,  1914,   415,  1150,   383,
  1583,   259,   438,   327,  1612,  1038,  2237,  1981,   570,   404,
   727,  1552,  2038,  1241,  1146,  1623,  1629,   513,   643,   730,
  1517,   837,   595,   642,  1641,   411,  1516,  1172,  2330,   260,
  2251,  2457,   737,  1019,  1071,   441,   377,   366,  2128,   725,
   771,   755,   535,  1271,  1616,   811,   594,  1713,   752,  1040,
  2197,  1696,   773,   411,   809,   228,  1062,   665,  1919,   242,
   592,   579,   368,   610,   329,   738,   788,   983,  1727,  1158,
  2056,   505,  1017,  1017,  1293,   290,   872,  2210,   586,   338,
  1094,  1778,  2286,   541,  1249,  1681,  1053,   859,   990,  1930,
   843,   504,  1638,   209,   784,   588,  2074,   441,  1253,  1101,
   451,  1554,  1175,  1192,  2198,   253,   788,  1747,  1637,   443,
   848,  1123,  2524,  2066,  2004,   764,   530,  1291,  1029,   849,
  1800,   599,  2267,  2326,   746,   772,  1821,  1774,  2506,  1011,
   670,  2198,   301,  1248,   352,  2349,   297,   765,   387,   626,
   372,   615,  1183,   381,  1785,   297,  1664,  1256,  2047,   468,
  2587,  1674,  2088,  1833,   778,   753,  1059,   438,   625,   800,
   461,  2103,   925,  1574,   861,  1202,  1361,  1184,  1598,   462,
  1049,   457,   543,   439,  2228,   620,   853,  2233,   818,  1632,
  1049,   611,  1120,   435,  2150,  2112,  1614,  1731,   888,  1818,
  1693,   404,   513,   515,  2131,   640,  1651,   860,  2265,   551,
   822,  1090,  2265,   564,   793,   551,  2472,   832,  1867,   301,
   871,   920,  1113,  1219,  2326,  1813,   800,  1064,   389,   341,
  1815,   561,  1623,  2287,   678,  2498,   622,   288,  1916,   611,
   680,   906,   394,   552,   554,  1892,   842,  1734,   539,  1147,
  2087,   244,  2340,  1318,   811,  1274,  2418,  1061,   806,   844,
  1732,   808,  1388,   721,  1134,   569,  1851,   348,   448,  2174,
   885,   467,  2299,   577,  1889,  1137,   656,   455,  2481,  2088,
  1228,  1331,  1896,   314,  2216,   481,  2621,   708,  1237,  1753,
  1143,   706,  1880,  2182,   929,   370,  1668,   802,  1629,   561,
   683,  1277,  2198,  1797,   927,   566,  2110,  2691,  2593,   545,
  2466,  2230,  1159,  1324,  2325,   243,  1119,   827,  1344,   315,
   411,  1708,  2149,  1348,   690,  1796,  1638,   491,   845,   424,
  1767,  1090,  1270,   487,   939,  1199,  2154,   644,   796,  2243,
   491,   331,   591,   728,   424,   360,   991,   892,  1789,   552,
  2548,   542,  2380,  1697,  1822,   298,  1203,  1783,  1891,  1686,
  1904,   595,   661,  2427,   857,   426,  2426,   838,   408,  1426,
   494,   398,  2423,   534,   682,  2395,  1983,  1270,  2235,  2163,
  2250,   661,  1827,   679,   497,   868,  2404,  1668,  1771,   817,
  1657,   430,  1211,   585,   381,  1103,  2419,   379,   508,   876,
  2629,   830,  1670,   806,  2669,   820,   823,   480,  1990,   609,
   691,  2500,  2855,  2752,   587,   741,  1824,   379,  1161,   338,
   277,  1219,  2776,  1949,   847,   866,  2599,  1334,  1225,  1123,
  1170,  1736,  1388,  1223,  1379,   654,   545,  1099,  1372,   592,
   960,  1975,  1757,   569,  2193,   584,   410,  2012,  2019,   583,
  1306,   443,   484,  2016,  1745,   890,  2513,  2058,  1133,   616,
   587,   698,  1324,   679,   770,  2705,  1946,  1106,  1183,  2591,
  1138,  1791,   551,   932,  1417,  1113,   734,  1316,  1241,   592,
  2190,  1728,   886,   344,  1321,   446,  1905,   694,   390,  1334,
   437,  1185,  2790,  1333,  1862,   601,   265,  1419,  2635,  2813,
  2809,   740,   338,  1349,  2475,   396,  2281,  2239,   485,   874,
  2651,   388,  2016,  1343,  2041,   761,  1178,   518,  2507,   392,
   700,   710,   616,  1429,  2673,  1349,  2346,   945,  2729,   604,
  2023,   480,   581,  1418,   879,  2552,  2756,   487,  2366,   342,
  2366,   529,  1768,  1446,  2915,  1020,  2090,  2246,   436,  1929,
  2146,  1369,   493,  1262,   865,  1429,  2254,  2729,   456,   686,
  2606,  2370,  1392,   616,  1363,  2114,  1363,   529,  1806,   338,
   317,   493,  2081,   682,   283,   611,  1919,   909,  2420,  2329,
  2611,   771,  2382,   436,  1832,   856,   872,  2392,   640,   447,
  2038,   636,   490,  1196,   801,  2578,  1406,   776,  1771,   403,
  1398,   435,  1492,  1792,  2619,   862,  1468,  1843,   972,  2544,
   750,  2420,  1468,   597,  1013,   713,  1221,   591,   981,   455,
   902,  2857,  2904,  1190,   907,  1160,   453,   453,   809,  1894,
   648,   584,  1288,  2770,  1028,  1820,   783,   895,   526,   651,
  1858,   726,   739,   330,   518,  2736,   742,  2860,  2053,   625,
  1036,  2617,  2860,  1066,  2089,   622,  1514,  1780,  2646,  2206,
   410,   642,  2899,  1556,   992,   436,  1980,   321,  2482,   513,
  1865,  2105,  2017,  1500,  2335,  2482,   327,   530,  1261,  1008,
  1029,  1208,   462,   444,  1397,   927,   512,  2875,  2128,  1447,
  1268,   504,  2033,  2191,  1041,  2191,  1241,  2178,   919,   657,
   945,   466,   708,   961,  2397,   826,  1353,   617,  1067,   898,
  1490,  1344,  2031,   950,  2398,  2498,   614,  3027,  1067,  2039,
  1518,  2094,  1542,  2138,   607,  1277,  2003,   735,   615,  1547,
  1243,  1227,  2028,  1903,  1060,  2212,  1014,   524,  1856,   647,
   547,   647,   984,   542,   733,   975,   532,  2256,  2237,   463,
  2758,  2673,  2118,  1353,   962,   379,  2924,  1977,   927,  2474,
  1463,  1241,   627,  2662,  2463,  2070,  2977,   363,   924,  2844,
   309,   284,   967,   417,  2276,   404,   644,  1960,  1295,  1290,
  1519,   803,   652,  2492,   366,  1523,  1246,   843,   842,   649,
  2869,  2571,  1481,  2954,  1071,  1276,  1091,   489,   412,  1069,
   619,   527,  1367,   346,  1299,  1981,  1954,  2804,   636,  1388,
  2169,   520,  2173,   917,  2160,  1264,   796,   840,   564,  2081,
   798,   983,  1993,   639,  1243,   960,  1889,  2568,  1452,   778,
  3153,  2476,  3189,   556,  2191,   959,  1364,  1294,  1313,  1574,
  2576,  1314,  1511,   426,  2214,  1001,  1603,  2186,  1268,  1268,
  2022,   662,  3039,   584,   426,   573,   810,  1482,  3059,  1114,
  2496,  2319,  1392,  1994,  1467,   296,   642,   487,  1129,  2851,
  2627,   350,  1065,  1073,  3021,   664,   486,   435,  2101,  1994,
   662,   429,  1516,   431,  1058,  1368,  2472,   773,   377,   751,
  1429,   695,   532,   701,  2509,   480,  2945,   507,  2352,   322,
  1531,   637,   498,   819,  2041,  2299,  1372,  1156,  1574,  2640,
  2042,   477,  2480,   951,   476,  2115,   799,  1133,  1356,  2004,
  1516,  3001,   879,  1383,  2046,   849,  1980,  1544,  2333,  2239,
  3025,  1061,  2695,  1344,  2656,   675,   484,   675,  1328,   833,
   626,   396,  1350,   379,   974,  2278,   685,   806,  2589,  1156,
  1601,  2359,   663,  3126,  1567,  1440,  1362,  3183,  1627,   989,
  2084,   716,   431,   384,  1624,   475,  1653,  2530,   839,   392,
   545,  2301,  2521,  2303,  1314,  2858,  2840,   833,  2002,  1299,
   768,  2280,   662,  1156,  2300,  3366,  2638,  1435,  1979,   686,
  2382,  2022,  2937,  1318,  2719,  1574,   775,   882,   656,   690,
  2863,   695,  2693,  2322,  2654,  2060,   563,  2116,  1349,  2187,
   709,   601,   773,   992,  3059,   561,  2076,  2122,  1452,   492,
  2017,  2291,  2861,  3108,  2072,  1457,  2590,  1115,  1146,  1631,
  1435,   645,  2170,  1339,  2102,  2626,   696,   692,   788,   829,
  2740,   592,  1551,   491,  1432,   592,  2174,  2686,  1657,   437,
  3268,  1489,  1003,  1017,  2305,  1567,  2222,   788,  2236,   661,
  1711,  2217,  2728,  2316,  2960,   359,  1346,  3150,   804,  1607,
  2019,   821,   828,  1024,  2992,  1000,  2411,   677,  2691,  2136,
   838,  3211,  2400,  2691,   732,  1105,  1559,  1027,  1649,   705,
  1399,   894,  1505,   705,   893,  1012,  1463,  1357,  1618,  1564,
  1719,  3147,   669,   415,  2421,  1174,  1118,   598,   580,  2657,
  3032,  1165,  3472,   505,  2997,  1474,   683,   400,  3432,  3451,
   446,  2222,  2772,   675,   928,   574,  1369,   803,  2678,  2177,
  1013,  3168,   882,  1171,  2137,  1079,  2828,  1176,   887,  2836,
  2649,  3055,  2404,  3124,   906,   716,  1578,  1019,  1017,   533,
  2288,   696,  3192,  1573,  1402,  1027,  1604,   624,   822,  1724,
  1208,  2824,  1154,   476,  2999,  1648,  1656,  1495,   403,  1639,
   677,  2182,  2211,   580,  3076,  2748,  1601,   591,  1516,  1638,
  1156,   728,  1194,  3210,  2814,   880,   611,  2470,  1654,  1515,
  2762,  1027,   409,   730,  1453,  3113,  2296,   719,   719,  1462,
  1502,   867,  2174,   866,  2268,  1414,  1026,   630,   581,  1776,
  2841,  2115,  1536,  2102,  2488,  1419,   702,  2445,  1720,   637,
  1178,  1047,   855,  1409,  2451,  2460,  3013,   603,  1153,  2426,
  3197,  2752,  1096,   683,   840,  1665,  1200,  2506,  2252,  1474,
  2553,  3352,  1542,  2806,  2449,  2554,   377,   737,  1619,   646,
  2916,  1115,   756,   472,  1160,  1075,  1756,   744,  1257,   948,
  2311,   693,  3573,  2338,  2940,  1641,  1661,   629,   689,  1755,
  3300,  2301,   538,  2796,   554,  2923,  3087,  2892,  2931,   701,
  3528,   387,  1473,  2950,   435,  2247,   517,  1413,  2551,  1440,
   593,  2912,  2763,   551,   631,   911,   870,  1424,  2514,   943,
  2912,   883,  3070,  1073,  2978,   918,  1223,   521,  3164,  1859,
  1203,  2614,  2387,  1505,   391,   632,  1425,  1171,   742,  2201,
  2469,   756,   647,  3164,  2190,   781,   438,  3314,  2183,  3437,
  3313,   633,   861,   535,  1204,  1823,  2610,  2362,  2978,  2178,
  3344,  1137,  1768,   735,   782,   864,   976,   881,  1212,  1299,
  1801,  1476,  3438,   777,  3195,  1251,  3194,  2650,  1208,  2816,
  1494,   530,  1477,  2952,   752,  1738,  2430,  1428,  2359,   557,
   491,  1219,  2958,   960,  1192,   733,  1790,  1602,  3236,   471,
  1138,  1725,   893,   556,  1569,  3337,  1305,   622,  3196,  1909,
  2210,  1560,  1610,  1206,  1691,  1744,   901,   957,  1162,  1465,
  1857,   431,  2600,  3332,  2811,  3272,   762,  1077,  1688,  2513,
  1621,   770,  2329,  1101,  1329,  1689,  1329,   747,   647,  1270,
   766,   572,  2993,   951,   472,  1520,  1581,  1121,  2260,  1747,
  1809,  1102,  3005,  1096,  1231,  1620,  1232,  1231,  2648,   631,
  2344,   745,  3003,  2643,  1243,  1751,  3006,   923,   893,   335,
  3072,  1914,  2415,  3473,  2274,   436,  1921,  3077,  1558,   723,
   957,  2221,  2562,  1894,   771,  1509,   723,  1561,  1163,  1589,
  2949,  1920,  1101,   643,  1775,   781,   978,   890,  3251,   720,
  3404,   569,  3437,  1226,   543,   870,  1746,  1480,  1177,  1221,
  1767,   870,   547,   642,  2387,  1137,   958,   966,  1854,   659,
  3317,   925,   572,  3317,   755,   762,  2668,  1946,  3057,  1569,
  3008,  1639,   929,  1147,  1361,  2351,  3087,  2308,  2718,  1178,
  1863,   663,  3021,  1557,  2771,  3330,  3738,  3581,  1189,  1024,
  1156,   561,  1579,   554,  1984,   519,   807,   764,  3622,  1618,
  1769,  1614,  2711,  1536,  1656,   647,  1921,   795,  1516,  1886,
  2952,  1881,  1672,   823,  1126,  1311,  1622,   878,  2745,  2623,
  3095,  3145,  1179,  1116,  1813,  1110,  1887,   685,  2653,  1818,
   563,  1650,  2785,  2654,  2823,  1269,  1191,   674,  1600,  1898,
  3881,  2625,  3506,  2316,  1279,  1685,  3427,  3558,  3194,  1685,
  2738,  1259,  1695,  2301,  1260,  2515,  2389,  3076,  3076,  1263,
   902,   647,  1657,   973,  1792,  3738,  3186,  1656,  2777,   697,
  1645,  1610,  3138,  1150,  1294,  1037,  3420,   748,   505,  1015,
  1680,   642,   494,  2526,  2757,  1602,  1532,  2439,  2440,   756,
  4014,  2572,  1825,  1576,  3542,  1203,  1278,  1558,  3923,   503,
  3901,  1775,  3172,  1025,   458,   596,  1530,  1728,  3373,   649,
  3458,   937,  3600,  1215,  3442,   511,  3174,   834,  3207,   680,
  1164,  1333,  1297,   831,  2787,   989,  2320,   847,  3562,  1229,
  1717,  3554,  2476,  1799,  1182,  2768,  3194,  1822,  1211,   993,
  3464,  2584,  2585,  1211,  2491,  1803,  3768,   711,   517,  2597,
  1551,  1343,  2473,  2485,  1318,  1180,  2515,   674,  3830,   517,
  3371,   848,  3857,  1947,  2724,  1700,   855,   516,  2698,  2864,
  3852,   974,  2515,  1599,  3622,  1580,  1298,  1539,  1838,   676,
  1679,  1584,  1584,   942,  2747,   818,   543,  2393,  1324,  1582,
  4106,  1582,  1729,   668,  1576,  1638,  3039,  1702,  1890,  2011,
  3248,  2515,   803,  1945,  3268,  2406,  1594,   459,  3961,   799,
  3179,   699,  2743,   667,  1328,   932,  3870,  3707,   541,   537,
  1984,  3715,  1716,  3324,  1769,  2492,  3791,  3299,  3666,  3187,
   825,  1582,  2615,   576,  1625,   953,  3453,  2860,  2890,  1970,
  1888,  2554,  1716,   933,  3703,  2753,  2847,   664,  1316,  1334,
  3254,  1216,  1836,   795,  2628,  2545,  2090,   868,  1345,  1270,
   596,   596,  1328,   531,  3759,   861,  1761,  2578,  3294,  1328,
  3201,  2049,  1058,  1349,  2639,  2903,  1572,   710,   473,  4041,
   687,   667,   714,   681,  1374,   731,  2913,   524,  3214,  1910,
  2422,  2782,  3563,  1789,  1329,  1762,  2798,   557,  3869,  2691,
   738,  1868,  1093,  3316,   558,  3625,  1194,   697,  2596,  1231,
   728,   950,   714,  3599,  2125,  1334,  2905,  1035,  2790,  3309,
  3517,   687,  3406,  2674,  3378,   633,  2567,  1810,  2469,  1950,
  2003,   681,   808,   871,  1628,  1884,  1713,   808,  1276,  3635,
  3349,  3006,  1763,  1906,  2955,  1241,   561,  1765,  3206,   850,
  1292,  1965,   987,  1757,  1631,  1229,  1777,   707,  1015,  2157,
   613,  2994,  1367,  3907,  2814,   481,  3928,   704,  2969,  1904,
  3343,  3640,  2083,   890,  1086,   448,  1281,   565,  3237,   630,
  2686,  2920,  3754,  2930,  1256,  1373,  3405,   725,  2615,   724,
  4290,  4290,  4290,   618,  2689,  1679,  1262,   710,   600,  1286,
  2993,  2860,  1369,   570,  1296,  1120,  1241,   711,  1241,   545,
  3396,  4126,  3658,  1260,  1112,  1244,  1815,   416,  1364,   715,
  3427,  1380,  1505,   700,  1272,   912,  1505,  1327,  1969,  1505,
  1259,  2580,  1261,  3420,   551,   871,  2048,  2717,  1978,  1693,
   982,   741,  1269,   406,  1710,  1783,  2977,   712,   915,  1674,
  1269,  1274,   607,  2683,  3452,  1462,  1462,  1375,  1271,  1130,
  3903,   741,   700,   910,  2173,  1275,  2658,  3855,  3839,  2668,
  3473,  2153,  1937,   572,  1868,  1287,  1074,   450,  2649,  2656,
  1325,   987,  3063,  2608,  1335,  1807,  1325,  1238,  1974,   766,
  1946,  1808,  1077,   852,  2101,  1447,  3013,  3877,  2641,  2958,
  1679,  2955,  3781,  2819,  2903,   627,  2645,   879,  2823,  1857,
  3513,  1082,  3323,  1399,  3327,  1976,  1344,   827,  2765,  1050,
  2648,   454,  2622,  3706,  1987,   624,  1325,   830,  2963,   746,
  1813,  4428,  4428,  3286,  3806,  1159,  3649,  1137,  2916,  3518,
  2642,   511,   449,  1262,  2637,   580,  1496,  2987,  4241,  3473,
  3000,   911,   668,  2965,   865,   668,  2004,  3308,  2607,   564,
  3087,  2974,  2105,  1017,  3989,  2058,  1356,   918,  2191,   364,
  1285,  2183,  2825,   862,   738,   578,  1060,   757,  2018,   504,
   732,  2796,  2570,  2856,  1061,  1008,  1365,  1283,  2039,  3548,
  2193,  2038,  2830,  1338,   595,  2112,  3503,  1278,  1750,  1415,
  1850,  3434,  3764,   737,   874,  1452,  1729,   595,  3714,  2595,
  2868,  1983,  3543,  3129,  3369,  2124,  3181,  1168,  3030,  1354,
  1569,  1326,  1569,  1569,  1569,   635,  1569,   473,  1569,  2625,
   519,  3728,  1569,  1867,  3560,  1377,  3993,   746,  3995,  2855,
  2855,  1149,  4037,  1149,  3149,  1918,  2859,  2208,  3111,  1889,
  3601,  3775,  1748,  1372,  2205,   560,  3010,  1330,  1329,  2929,
  3753,  3104,  3175,  3810,  4025,  3189,  1331,  1129,  3791,  1378,
  1389,  1021,  2896,  4092,  4005,  3394,  3007,   852,  3119,  3008,
  2727,   886,  2045,  4006,  2099,   880,  3191,  1878,  2666,  1368,
  2872,   806,  1895,   770,  3451,  1895,  2283,  2347,  1041,  4285,
  1902,   673,  1121,   889,   784,   886,   961,  2059,  1036,  1351,
  2747,  1335,  1757,  1075,  3640,   595,   799,  1460,  2060,  1766,
  1907,  4030,  2030,   650,  1793,   646,   809,   894,  1848,  2041,
  1037,  3546,  1601,  2095,  2893,  1352,  3252,  3518,  1397,  1774,
  3877,   746,  2121,   931,  3867,   746,  1962,  2037,  2056,  2960,
  3040,   953,   809,  1598,   527,  1323,  1405,  1499,  3113,   475,
  3225,  1967,  3225,   966,  2844,  2971,  3975,  1395,  1136,  3572,
  1929,  1090,  3136,  2987,  2262,   974,  1136,   793,  3102,  1471,
   679,  1181,  1621,  2851,  2310,  1850,  2226,   764,  1103,  1414,
  2871,  3895,  4521,  3240,  3252,   905,  4098,  2437,  3003,   542,
  1156,  2269,   913,  1957,   776,  1403,  2437,  3579,  3891,  1187,
   808,   659,  3248,  1320,  3484,   770,  3492,   653,  1546,   446,
  2226,   936,  3603,  2929,  2767,  3587,  2843,  1975,  1949,   953,
  2888,   497,   776,  1107,  2857,  1432,  1353,  2965,  1508,   894,
  3679,  1157,  4704,  2868,  1485,   752,  1417,   755,  3488,  1196,
  3969,  2023,  3788,  3572,   678,  4366,  2242,  1908,  2110,  1432,
  4217,   882,  2152,  2842,  2266,  2111,  4155,  1166,  1422,  2020,
  3965,  1177,  2127,  1077,  1846,   955,  2110,  1089,   944,  1865,
  1389,   901,  1987,  2976,  2363,  2001,  3586,   681,  3304,  3921,
  3632,  1097,  3143,  4046,  1219,  3558,  1886,  1450,  3719,  1444,
  3208,  2798,  2803,  1421,  1986,  2017,   826,  1335,  2289,   816,
  3641,  4261,  3321,  2814,  4350,  1428,  4259,  2451,  1121,  1878,
  1458,  3664,  3740,  3227,  1532,   425,  1985,   677,  3149,   912,
  2935,  2941,  1827,   487,  4016,  1988,  2292,   813,  3766,   539,
  1587,  1397,  3062,   928,  1943,   724,  2162,   927,  3059,  2509,
  1221,   927,  3036,  4430,  4088,   705,  3786,  1010,  2393,  3203,
  1828,  1814,  3098,  1837,   836,   690,  1517,  1889,  1926,  1369,
   821,  1247,  3308,  1993,  2348,  1926,  1183,   606,  1997,  1016,
   816,  2273,   639,  4462,  3795,  2220,  1529,   769,  3098,  1997,
  3737,   685,  1573,  1528,  1424,  3803,  4729,   692,  2379,  4069,
  1928,   937,  1449,   913,  2212,   500,  4016,  2041,   907,  2060,
  1397,  2310,  2924,  2022,  2007,  3285,  3617,  1600,  2324,  1882,
   791,   929,  2964,  1157,  2243,   638,   974,  2193,  1697,  1697,
  1251,  3124,  1959,  1870,  3239,  1212,   555,  3107,  2025,  3221,
  2073,   716,  2166,  3828,  2876,  1459,  3886,  3348,  2235,  3285,
  1890,  1651,  4061,  1169,  1613,  3713,   618,  3354,  1223,  1577,
  1208,   632,  4042,   961,   850,  1168,  4172,  3935,  1155,  3027,
   961,  3390,  3390,   736,  1922,   850,  1895,  1897,  2312,   626,
  2982,   940,  3378,   571,  1212,   722,  3405,  1916,  4128,  1231,
  2234,  4669,  1483,  1212,  3032,   579,  3032,  1110,   647,  1576,
  3153,  1143,  2039,  3401,  4350,  4172,  2271,   839,  3009,  3045,
   874,  3671,  3165,  1458,  1230,  4133,  3886,  1672,  3444,   948,
  1943,  3137,  2060,  2091,  4119,  1180,  3829,  4530,  1974,  1434,
  1131,  2480,  3122,  2045,  3887,  1172,  2826,  2902,  2224,  3033,
  4980,  1602,  1916,  2965,  1586,  1147,  3921,  4378,   996,  3033,
  3324,  1514,  1603,  1238,  1958,   855,   950,  1611,  3815,   803,
   635,  3042,  1030,  4462,  1722,  2919,  1130,  2248,  3070,  1013,
  1225,  3919,   661,   509,  2286,   526,  1967,  2004,  3130,  2000,
  2295,   941,  1450,   808,  3747,  3954,  3768,  2008,  2261,   704,
  1512,  3318,   634,   517,  1628,   816,  3230,  3497,  3754,  1294,
  2383,   938,  3960,   980,  1301,   655,   744,  1951,  3824,  1145,
  2374,  2414,  2119,  1263,  1026,  1515,  2246,   632,  2320,  1512,
   521,  1636,  2254,   632,  4521,  4311,  3191,  3012,  2571,  2422,
  1475,  1140,   583,  3219,  2322,  1064,  1135,  3991,  3028,  1195,
  4636,  1980,  2421,  3114,  4873,  1056,  3886,  4636,  3407,  1670,
  2321,  3129,   751,  4258,  2989,  1322,  2389,  1041,  3818,   879,
  1554,  3548,  1550,  1699,  3446,  3525,  2536,  1741,  1994,  2496,
  2143,  1488,  3515,   722,  4335,  1028,  2036,   765,  3297,   827,
   825,  3912,  1317,   827,  4257,  3212,  4051,  3492,  5118,  3562,
  4026,  1533,  1687,  4353,  4678,  2544,   756,  3879,  2185,  1955,
  2159,  2193,  4099,   993,  2049,  2061,   965,  3458,  1031,   574,
  3161,  4548,  4136,  3216,  1257,   478,  4268,  2362,  1705,  3970,
  4702,  3562,  4575,  1166,  4382,  1952,   962,  2003,  4431,   447,
  3037,  2146,  3239,  2321,  2022,   826,  1746,  4257,  1510,   759,
  2329,  1977,  1635,  2308,  2306,  2438,  2203,  2171,  4575,  1577,
  3270,  1510,  2359,  3164,   739,  2083,  4280,   916,  3882,  3164,
  2096,  3894,  3190,  1331,  2579,  1358,  4864,  3453,   883,   856,
  2357,  2184,  1497,   581,  1170,  2394,  2002,  4326,  1337,  3054,
  2491,  1809,  1809,   885,  2071,  1580,  3898,  2386,  3877,  1484,
   979,  1161,  3952,  4103,   913,  1751,  4623,  3179,  4079,  2082,
  3627,  3506,  4676,  2078,  3514,  4492,  1601,  1678,  2565,  1021,
  2038,  4438,  1685,  2237,  4321,  3955,  2056,  1102,  3137,  1327,
  1604,  2198,  4724,  2363,  1761,  4607,  2321,  1282,  4337,   889,
  3930,  2506,  2099,  1207,  3521,  4195,  3618,  3521,  4464,  4192,
  4380,  4479,  1248,  5031,  2207,  2066,  1736,   737,  1589,  3628,
  1523,  3295,   891,  1611,   924,  1224,  3404,  2100,  5033,  2596,
  3329,  4001,  4813,  3545,  3248,  4823,  3545,  2377,  3356,  4954,
  1222,  4070,  3338,   890,  3325,  2505,  1110,  3405,  4818,  2006,
  1299,  1346,  1785,  3181,  3967,  3573,   926,  1193,  2506,  2507,
  1387,  1296,  2083,  2515,   862,  2231,  1038,  1267,  2357,  1263,
  3696,   851,  3573,  3067,  2075,   509,  1355,  2003,  5087,   665,
  4412,  3075,  1819,  1766,   623,   509,  4868,   849,  1264,  3989,
  4543,   909,  2093,   541,  3578,  4890,  4195,   989,  1755,  4007,
  2129,  1218,  4722,  2126,  1097,  2126,  1119,  2381,  5348,  2210,
  2149,  3401,  2252,  3431,  3410,   884,  2446,   745,  4887,  1599,
  2373,  4059,  1264,  4556,  2606,  3221,  2617,  2608,  2384,  1120,
  3737,  1087,  5133,  2409,  1768,  2382,  3608,   756,  2382,  4325,
  2524,   940,  3274,  1010,  2390,  3318,  2450,  3300,  1577,  1727,
  3203,  2643,  2679,  4965,  2618,  4165,  4165,  3277,  1568,  1574,
  2165,   633,  4836,  1076,  3382,  3400,  3645,   636,  4080,  3469,
  4555,  1329,  4606,  3314,  1720,  4101,  4598,  1107,  1641,   760,
  1544,  3248,  3761,  1803,  1096,  1749,  4765,  3132,  2089,  2302,
  3231,  3237,  2706,  4314,  3255,  2637,  4299,  1889,  1889,  1889,
  2076,  2559,  1889,  2559,  1747,   778,  1541,  1636,  1599,  3608,
  2267,  1746,  2191,  1237,  2429,  3309,  2307,  3250,  3853,  3175,
   688,  1564,  4111,  2456,   885,  1390,  2412,   896,   684,  4344,
  3801,  4874,  3516,  4102,  4812,  3801,  2258,  3375,  2579,  1905,
  1905,   780,  4354,  1868,  1905,   684,  1837,  2513,  1638,  1109,
  1610,  1905,  2177,  1850,  4310,  2218,  1135,  4292,  1653,  2467,
  3763,  1390,  3633,  3851,  2310,  2119,  1613,  4240,  2514,  4916,
   789,  1665,  1117,  3823,   715,  1111,  1037,  1582,  3386,  1292,
  4234,  2746,  1607,  3462,  4297,  2103,  3929,  2439,  3525,  1056,
  3490,  1661,  1817,  1815,  2320,   943,  1679,  2451,  3643,  1377,
  1777
]

In [132]:
M = 8
T = 200.0
dt = 0.25
N, tau = fed_tau_by_process_time(T, M, dt, True)
print N, tau[0:10], 'speedup:', (T/dt)/(M*N)

17 [ 0.17594662  0.99258695  0.1247977   0.23457392  3.84376291  0.13548686
  0.35470921  0.15803631  0.65097318  0.12279619] speedup: 5.88235294118


In [123]:
N, tau = _fed_tau_internal(2500, 1.0, 0.25, True)
T = sum(tau)
print N, T

2500 521041.666666
